In [7]:
import sys 
import numpy as np 
sys.path.append('/home/ak/Documents/Research/QFPaper/Code/')
import hmm 
import pandas as pd
import plotly
import plotly.plotly as py
import seaborn as sns
from hmm import hmm_calibration
from observation_models import *
import matplotlib.pyplot as plt 
%matplotlib inline 
import copy

prng = np.random.RandomState(10)
n_components = 3
startprob = prng.rand(n_components)
startprob = startprob / startprob.sum()
transmat = prng.rand(n_components, n_components) #random starting point
transmat /= np.tile(transmat.sum(axis=1)[:, np.newaxis], (1, n_components))#normalising so all entries sum up to 1

sigmas = [ .3, 0.8, 1][0:n_components]
lambdas = [0.5, 5., 0.8][0:n_components]
weights = [0, 0, 0][0:n_components]
init = 1. / n_components

tpm_init = np.full((n_components, n_components), init)
priors = {'sigmas':sigmas[0:n_components], 'lambdas':lambdas[0:n_components], 
          'weights':weights[0:n_components], 
          'tpm': tpm_init, 'pi': startprob}

sample_size = 500

#generate state and observation data from the \"right\" distribution\n",
states = hmm_calibration.sample_states(startprob, transmat, prng, sample_size = sample_size)
#define the calibrator and model 
observation_data = Exp_UniGauss.sample_data(states,lambdas, sigmas, weights, 0, rng=prng)
data_set = pd.DataFrame()
data_set['tau'] = observation_data[0]
data_set['price_change'] = observation_data[1]
update_tag = 't'

obs_model = Exp_UniGauss(n_components, epsilon=0)
the_hmm = hmm_calibration(obs_model, n_components, max_iter=10) 
result = the_hmm.run_hmm(data_set, update_tag=update_tag, initialise_method_tag=None, priors=priors, rng=prng)
print "transmat:" 
print transmat, "\n" 

sigma: 

[0.3, 0.8, 1] 

transmat:
[[ 0.50866113  0.33863492  0.15270396]
 [ 0.17563367  0.67440608  0.14996025]
 [ 0.05114943  0.39682856  0.55202201]] 



In [8]:
for step_obj in result['step_calibrations']: 
    print step_obj.tpm_, "\n" 
   

[[ 0.33988139  0.35939499  0.30072362]
 [ 0.26055825  0.44995867  0.28948308]
 [ 0.26653298  0.40848289  0.32498414]] 

[[ 0.36745028  0.35616292  0.2763868 ]
 [ 0.22066995  0.52937676  0.24995329]
 [ 0.22699705  0.4493269   0.32367604]] 

[[ 0.4022215   0.34033007  0.25744843]
 [ 0.20211347  0.57679905  0.22108749]
 [ 0.20023188  0.46854696  0.33122116]] 

[[ 0.43463482  0.32290367  0.24246151]
 [ 0.1950036   0.60417277  0.20082363]
 [ 0.17817225  0.47685844  0.34496931]] 

[[ 0.46077828  0.30825193  0.23096979]
 [ 0.19343354  0.6203978   0.18616865]
 [ 0.15788663  0.47999712  0.36211625]] 

[[ 0.48043373  0.29706704  0.22249923]
 [ 0.19439983  0.63054932  0.17505085]
 [ 0.13893945  0.48052955  0.38053101]] 

[[ 0.49480531  0.28876745  0.21642725]
 [ 0.19647562  0.63730687  0.16621751]
 [ 0.12161718  0.47964176  0.39874106]] 

[[ 0.50526185  0.28259167  0.21214648]
 [ 0.19898773  0.64208595  0.15892633]
 [ 0.10618203  0.47799324  0.41582473]] 

[[ 0.51290984  0.27792486  0.2091653 ]
 

In [19]:
tpm_hat = np.zeros
gamma_hat = np.zeros(n_components) 
tpm_hat = np.zeros((n_components, n_components)) 

for i in range(sample_size-1):
    tpm_hat[states[i], states[i+1]] += 1 
    gamma_hat[states[i]] += 1 

tpm_hat /= float(sample_size)
gamma_hat /=float(sample_size) 

for i in range(n_components): 
    tpm_hat[i,:] /= gamma_hat[i]


print "empirical:\n", tpm_hat,"\n"
print "original:\n", transmat 
    



empirical:
[[ 0.50793651  0.36507937  0.12698413]
 [ 0.21189591  0.6802974   0.10780669]
 [ 0.03846154  0.39423077  0.56730769]] 

original:
[[ 0.50866113  0.33863492  0.15270396]
 [ 0.17563367  0.67440608  0.14996025]
 [ 0.05114943  0.39682856  0.55202201]]
